<h1> Pruning with Pytorch </h1>
This was done with help of a tutorial and extensive short-term learning from documentation because coding for pruning wasn't taught in our institution, although I am familiar with the concept in theory.

In [54]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader
model = torch.load('./models/modelLeNet.pth')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
device
from torch.utils.data import random_split
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_size = 40000
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])
#print(train_size, val_size) #they are 40000 and 10000 respectively.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root="./data/", train=False, download=True, transform=transform)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
testloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


<h3>Loading ResNet and Pruning the said Model</h3>

In [55]:
import torch.nn.utils.prune as prune

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(pretrained=False)
model1 = resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model.load_state_dict(torch.load('./models/modelResNet.pth'))
pruning_ratio = 0.7
model1 = model
parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.fc, 'weight'),
    (model.bn1, 'weight'),

)
#print(list(model.named_parameters()))

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.5,
)
prune.remove(model.bn1, 'weight')
prune.remove(model.conv1, 'weight')
prune.remove(model.fc, 'weight')


Linear(in_features=512, out_features=10, bias=True)

<h4> Getting an idea of sparsity of the tensors, such that we can determine an appropriate prune ratio, if we were not already given the example figures of 50% or 70%

In [56]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv1.weight == 0))
        / float(model.conv1.weight.nelement())
    )
)
print(
    "Sparsity in fc.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc.weight == 0))
        / float(model.fc.weight.nelement())
    )
)
print(
    "Sparsity in bn1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.bn1.weight == 0))
        / float(model.bn1.weight.nelement())
    )
)
#print(list(model.named_parameters()))

Sparsity in conv1.weight: 53.47%
Sparsity in fc.weight: 44.26%
Sparsity in bn1.weight: 0.00%


<h4> Seeing if our serialized model has the same keys for required "weights", "biases", etc. for future use and predictions in tensors.

In [57]:
print(model.state_dict().keys() == model1.state_dict().keys())

True


This step finally saves the model

In [58]:
torch.save(model.state_dict(), 'models/prunedResNetModel.pth')

# Determining the accuracy 
Key point- Forming a generalized function for reusability


In [59]:


def evaluate(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    predictions=[]
    with torch.no_grad():
        for data in data_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(outputs.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy,predictions


In [60]:
model = resnet18(pretrained=False)
model1 = resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model.load_state_dict(torch.load('./models/prunedResNetModel.pth'))

<All keys matched successfully>

<h3>Accuracy of our ResNet model for the Validation set

In [61]:

accuracy,predictions = evaluate(model, val_loader)

print(f'Accuracy on the validation dataset: {accuracy:.2f}%')

Accuracy on the validation dataset: 84.19%


<h3> Accuracy of our ResNet model for the Test Set

In [62]:
accuracy,predictions = evaluate(model, testloader)
print(f'Accuracy on the test dataset: {accuracy:.2f}%')

Accuracy on the test dataset: 83.93%


We can safely say that these accuracies are pretty appealing.

# LeNet

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from LeNet import LeNet
model = LeNet()
model1 = LeNet()
model.load_state_dict(torch.load('./models/modelLeNet.pth'))
pruning_ratio = 0.7
model1 = model
for name, module in model.named_modules():
    # prune 20% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.5)
        prune.remove(module, 'weight')



In [64]:
for name, module in model.named_modules():
    # prune 20% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):    
        print(
            "Sparsity in {}: {:.2f}%".format(name,
                100. * float(torch.sum(module.weight == 0))
                / float(module.weight.nelement())
            )
        )


Sparsity in features.0: 50.00%
Sparsity in features.3: 50.00%
Sparsity in classifier.0: 50.00%
Sparsity in classifier.2: 50.00%
Sparsity in classifier.4: 50.00%


<h3> Accuracies for Validation and Test sets respectively wrt our LeNet model

In [65]:

accuracy,predictions = evaluate(model, val_loader)
print(f'Accuracy on the validation dataset: {accuracy:.2f}%')

Accuracy on the validation dataset: 61.64%


In [66]:
accuracy,predictions = evaluate(model, testloader)
print(f'Accuracy on the test dataset: {accuracy:.2f}%')

Accuracy on the test dataset: 60.88%


This step saves the model

In [67]:
torch.save(model.state_dict(), 'models/prunedLeNetModel.pth')

In [70]:
import os
initial_model_size = os.path.getsize('./models/modelLeNet.pth') / (1024 * 1024)  # in MB
print(f'Initial model size: {initial_model_size:.2f} MB')

# Calculate model size after pruning
pruned_model_size = os.path.getsize('./models/prunedLeNetModel.pth') / (1024 * 1024)  # in MB
print(f'Pruned model size: {pruned_model_size:.2f} MB')

Initial model size: 0.24 MB
Pruned model size: 0.24 MB
